In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from mpl_toolkits import mplot3d
sns.set()

# Load Data

In [ ]:
def load_data(data_location):
    return pd.read_csv(data_location)

In [ ]:
data_location = "data/denver_data.csv"
df = load_data(data_location)

In [ ]:
df.head()

# Create Series Data

In [ ]:
def find_unique_year_month_day(data):
    unique_year_month_day = []
    
    for date in data["datetime"]:
        year_month_day = date.split(" ")[0]
        if year_month_day not in unique_year_month_day:
            unique_year_month_day.append(year_month_day)

    return unique_year_month_day
        
def fill_missing_values(data):
    year_month_day = data["datetime"].values[0].split(" ")[0]
    
    hours = []
    for i in range(24):
        if i < 10: hour = "0" + str(i)
        else: hour = str(i)
            
        hours.append(year_month_day + " " + hour + ":00:00")
    
    return data.set_index('datetime').reindex(hours).fillna(0).reset_index()
        
    
def series(data, time):
    # returns 24 hours of series data for a given time.
    # time is in the year-month-day format with no hours.
    time_series = df.loc[df['datetime'].str.startswith(time)]
    time_series = fill_missing_values(time_series)
    
    return time_series

In [ ]:
unique_year_month_day = sorted(find_unique_year_month_day(df))

In [ ]:
series_data = pd.DataFrame(columns=df.columns)

for year_month_day in unique_year_month_day:
    series_data = series_data.append(series(df, year_month_day))

In [ ]:
series_data.head()

In [ ]:
def description_values(data):
    categorical_values = {}
    categorical_value = 1
    for weather_description in data["weather_description_Denver"]:
        if weather_description != '0' and weather_description not in categorical_values:
            categorical_values[weather_description] = categorical_value
            categorical_value += 1
    return categorical_values

In [ ]:
categorical_values = description_values(series_data)
categorical_values

In [ ]:
series_data["weather_description_Denver"].replace(categorical_values, inplace=True)

In [ ]:
series_data.to_csv("data/denver_series_data.csv", index=False)